In [1]:
# Uncomment to install/upgrade ras-commander from pip
#!pip install --upgrade ras-commander

#Import the ras-commander package
from ras_commander import *

2025-11-17 13:35:59 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


##### Optional Code Cell For Development/Testing Mode (Local Copy)
##### Uncomment and run this cell instead of the pip cell above

```python
# For Development Mode, add the parent directory to the Python path
import os
import sys
from pathlib import Path

current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent

# Use insert(0) instead of append() to give highest priority to local version
if str(rascmdr_directory) not in sys.path:
    sys.path.insert(0, str(rascmdr_directory))

print("Loading ras-commander from local dev copy")
from ras_commander import *
```

In [2]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from IPython import display
import matplotlib.pyplot as plt
import psutil  # For getting system CPU info
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import subprocess
import shutil

In [3]:
# Extract the Bald Eagle Creek example project
# The extract_project method downloads the project from GitHub if not already present,
# and extracts it to the example_projects folder
bald_eagle_path = RasExamples.extract_project("Balde Eagle Creek")
print(f"Extracted project to: {bald_eagle_path}")  


# Verify the path exists
print(f"Bald Eagle Creek project exists: {bald_eagle_path.exists()}")

2025-11-17 13:36:01 - ras_commander.RasExamples - INFO - Found zip file: C:\GH\ras-commander\examples\Example_Projects_6_6.zip


2025-11-17 13:36:01 - ras_commander.RasExamples - INFO - Loading project data from CSV...


2025-11-17 13:36:01 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.


2025-11-17 13:36:01 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----


2025-11-17 13:36:01 - ras_commander.RasExamples - INFO - Extracting project 'Balde Eagle Creek'


2025-11-17 13:36:01 - ras_commander.RasExamples - INFO - Project 'Balde Eagle Creek' already exists. Deleting existing folder...


2025-11-17 13:36:01 - ras_commander.RasExamples - INFO - Existing folder for project 'Balde Eagle Creek' has been deleted.


2025-11-17 13:36:01 - ras_commander.RasExamples - INFO - Successfully extracted project 'Balde Eagle Creek' to C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek


Extracted project to: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek
Bald Eagle Creek project exists: True


## Understanding Plan Specification in HEC-RAS

In HEC-RAS, each plan (`.p*` file) represents a specific hydraulic model simulation scenario. When working with RAS Commander, you can specify plans for execution in several ways:

1. **Single Plan**: Specify one plan by its number (e.g., "01")
2. **List of Plans**: Specify multiple plans as a list (e.g., ["01", "03", "05"])
3. **All Plans**: Execute all plans in a project by not specifying any plan or passing `None`
4. **Filtered Plans**: Select plans based on criteria (e.g., plans with specific flow conditions)
5. **Plan Path**: Specify the full path to a plan file instead of just the number

### Why Plan Specification Matters

- **Efficiency**: Run only the plans you need rather than recomputing everything
- **Organization**: Group related plans for batch processing
- **Automation**: Create workflows that process plans in a specific order
- **Resource Management**: Optimize hardware utilization for specific plans

### Best Practices for Plan Specification

- Use consistent formatting for plan numbers (e.g., always use two-digit strings like "01" instead of 1)
- Check available plans before attempting to execute them
- Organize plans by purpose to make selection easier
- Use descriptive short identifiers and plan titles to aid in selection

## Step 1: Project Initialization

Let's initialize the HEC-RAS project using the `init_ras_project()` function and explore the available plans.

In [4]:
# Initialize the HEC-RAS project
init_ras_project(bald_eagle_path, "6.6")
print(f"Initialized HEC-RAS project: {ras.project_name}")

# Display the current plan files in the project
print("\nAvailable plans in the project:")
display.display(ras.plan_df)

# Check plan details to understand what each plan represents
plan_details = []
for index, row in ras.plan_df.iterrows():
    plan_number = row['plan_number']
    
    # Get plan description if available
    description = None
    if 'description' in row:
        description = row['description']
    else:
        try:
            description = RasPlan.read_plan_description(plan_number)
        except:
            pass
    
    # Get short identifier if available
    short_id = None
    if 'Short Identifier' in row:
        short_id = row['Short Identifier']
    
    # Get geometry file
    geom_file = None
    if 'Geom File' in row:
        geom_file = row['Geom File']
    
    # Check if the plan has results
    has_results = False
    if 'HDF_Results_Path' in row and row['HDF_Results_Path']:
        has_results = True
    
    plan_details.append({
        'Plan Number': plan_number,
        'Short ID': short_id,
        'Description': description[:50] + '...' if description and len(description) > 50 else description,
        'Geometry': geom_file,
        'Has Results': has_results
    })

# Create a DataFrame with the plan details
plan_details_df = pd.DataFrame(plan_details)
print("\nPlan details:")
display.display(plan_details_df)

2025-11-17 13:36:01 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.rasmap


Initialized HEC-RAS project: BaldEagle

Available plans in the project:


plan_number unsteady_number geometry_number                     Plan Title  \
0          01              02              01  Unsteady with Bridges and Dam   
1          02            None              01                Steady Flow Run   

  Program Version Short Identifier                  Simulation Date  \
0            5.00     UnsteadyFlow    18FEB1999,0000,24FEB1999,0500   
1             NaN        SteadyRun  02/18/1999,0000,02/24/1999,0500   

  Computation Interval Mapping Interval Run HTab  ... PS Cores DSS File  \
0                 2MIN            1HOUR        1  ...     None      dss   
1                 2MIN              NaN        1  ...     None      dss   

  Friction Slope Method HDF_Results_Path Geom File  \
0                     2             None        01   
1                     1             None        01   

                                           Geom Path  Flow File  \
0  C:\GH\ras-commander\examples\example_projects\...         02   
1  C:\GH\ras-commander\examples\example_projects\...         02   

                                           Flow Path  \
0  C:\GH\ras-commander\examples\example_projects\...   
1  C:\GH\ras-commander\examples\example_projects\...   

                                           full_path flow_type  
0  C:\GH\ras-commander\examples\example_projects\...  Unsteady  
1  C:\GH\ras-commander\examples\example_projects\...    Steady  

[2 rows x 27 columns]

2025-11-17 13:36:01 - ras_commander.RasPlan - WARNING - No description found in plan file: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01


2025-11-17 13:36:01 - ras_commander.RasPlan - WARNING - No description found in plan file: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p02



Plan details:


Plan Number      Short ID Description Geometry  Has Results
0          01  UnsteadyFlow                   01        False
1          02     SteadyRun                   01        False

## Step 2: Sequential Execution of Specific Plans

Let's execute specific plans in sequence using `RasCmdr.compute_test_mode()` with a list of plan numbers. This approach allows us to run only the plans we need, in the order we specify.

In [5]:
print("Executing specific plans sequentially...")
print("This may take several minutes...")

# Define the plans to execute
specific_plans = ["01", "03"]
print(f"Selected plans: {', '.join(specific_plans)}")

# Record start time for performance measurement
start_time = time.time()

# Execute specific plans sequentially
execution_results = RasCmdr.compute_test_mode(
    plan_number=specific_plans,
    dest_folder_suffix="[SpecificSequential]",
    num_cores=6, 
    overwrite_dest=True
)

# Record end time and calculate duration
end_time = time.time()
sequential_duration = end_time - start_time

print(f"Sequential execution of specific plans completed in {sequential_duration:.2f} seconds")

# Create a DataFrame from the execution results for better visualization
sequential_results_df = pd.DataFrame([
    {"Plan": plan, "Success": success, "Execution Type": "Sequential"}
    for plan, success in execution_results.items()
])

sequential_results_df 

# Ensure the 'Plan' column exists before sorting
if 'Plan' in sequential_results_df.columns:
    sequential_results_df = sequential_results_df.sort_values("Plan")
else:
    print("Warning: 'Plan' column not found in execution results.")

# Display the results
print("\nSequential Execution Results:")
display.display(sequential_results_df)

# Check the test folder
test_folder = bald_eagle_path.parent / f"{ras.project_name} [SpecificSequential]"
if test_folder.exists():
    print(f"\nTest folder exists: {test_folder}")
    
    # Check for results
    hdf_files = list(test_folder.glob("*.p*.hdf"))
    if hdf_files:
        print(f"Found {len(hdf_files)} HDF result files:")
        for file in hdf_files:
            file_size = file.stat().st_size / (1024 * 1024)  # Size in MB
            print(f"  {file.name}: {file_size:.1f} MB")
    else:
        print("No HDF result files found in the test folder")

2025-11-17 13:36:01 - ras_commander.RasCmdr - INFO - Starting the compute_test_mode...


2025-11-17 13:36:01 - ras_commander.RasCmdr - INFO - Creating the test folder: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek [SpecificSequential]...


2025-11-17 13:36:01 - ras_commander.RasCmdr - INFO - Copied project folder to compute folder: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek [SpecificSequential]


2025-11-17 13:36:01 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek [SpecificSequential]\BaldEagle.rasmap


2025-11-17 13:36:01 - ras_commander.RasCmdr - INFO - Initialized RAS project in compute folder: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek [SpecificSequential]\BaldEagle.prj


...

2025-11-17 13:37:36 - ras_commander.RasCmdr - INFO - compute_test_mode completed.


2025-11-17 13:37:36 - ras_commander.RasCmdr - INFO - 
Execution Results:


2025-11-17 13:37:36 - ras_commander.RasCmdr - INFO - Plan 01: Successful


Sequential execution of specific plans completed in 94.86 seconds

Sequential Execution Results:


Plan  Success Execution Type
0   01     True     Sequential

## Step 3: Running Only Plans Without HDF Results
An important use case is to identify and execute only those plans that have no existing HDF results. This approach can save time by avoiding redundant computations, especially useful when adding new plans to an existing project or after making limited changes.

Let's demonstrate how to:

- Use the `ras` object to identify plans without results
- Create a filtered list of these plans
- Execute only the missing plans

In [6]:
print("Identifying and executing plans without HDF results...")

# Use the ras object to determine which plans don't have results
plans_no_results = ras.plan_df[ras.plan_df['HDF_Results_Path'].isna()]['plan_number'].tolist()

if not plans_no_results:
    print("All plans already have HDF results. Creating a test scenario...")
    # For demonstration purposes, pretend some plans don't have results
    plans_no_results = ["04", "05"]
    print(f"Simulating no results for plans: {', '.join(plans_no_results)}")
else:
    print(f"Found {len(plans_no_results)} plans without HDF results: {', '.join(plans_no_results)}")

# Record start time for performance measurement
start_time = time.time()

# Execute only the plans without results
if plans_no_results:
    print(f"\nExecuting {len(plans_no_results)} plans without results...")
    execution_results = RasCmdr.compute_test_mode(
        plan_number=plans_no_results,
        dest_folder_suffix="[MissingPlans]",
        num_cores=6, 
        overwrite_dest=True
    )
    
    # Record end time and calculate duration
    end_time = time.time()
    duration = end_time - start_time
    
    print(f"Execution completed in {duration:.2f} seconds")
    
    # Create a DataFrame from the execution results
    missing_results_df = pd.DataFrame([
        {"Plan": plan, "Success": success, "Execution Type": "Missing Plans"}
        for plan, success in execution_results.items()
    ])
    
    # Sort by plan number
    missing_results_df = missing_results_df.sort_values("Plan")
    
    # Display the results
    print("\nExecution Results for Plans Without HDF Results:")
    display.display(missing_results_df)
    
    # Check the test folder
    test_folder = bald_eagle_path.parent / f"{ras.project_name} [MissingPlans]"
    if test_folder.exists():
        print(f"\nTest folder exists: {test_folder}")
        
        # Check for results
        hdf_files = list(test_folder.glob("*.p*.hdf"))
        if hdf_files:
            print(f"Found {len(hdf_files)} HDF result files:")
            for file in hdf_files:
                file_size = file.stat().st_size / (1024 * 1024)  # Size in MB
                print(f"  {file.name}: {file_size:.1f} MB")
        else:
            print("No HDF result files found in the test folder")
else:
    print("No plans without results to execute.")

2025-11-17 13:37:36 - ras_commander.RasCmdr - INFO - Starting the compute_test_mode...


2025-11-17 13:37:36 - ras_commander.RasCmdr - INFO - Creating the test folder: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek [MissingPlans]...


2025-11-17 13:37:36 - ras_commander.RasCmdr - INFO - Copied project folder to compute folder: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek [MissingPlans]


2025-11-17 13:37:36 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek [MissingPlans]\BaldEagle.rasmap


2025-11-17 13:37:36 - ras_commander.RasCmdr - INFO - Initialized RAS project in compute folder: C:\GH\ras-commander\examples\example_projects\Balde Eagle Creek [MissingPlans]\BaldEagle.prj


...

2025-11-17 13:39:14 - ras_commander.RasCmdr - INFO - 
Execution Results:


2025-11-17 13:39:14 - ras_commander.RasCmdr - INFO - Plan 01: Successful


2025-11-17 13:39:14 - ras_commander.RasCmdr - INFO - Plan 02: Successful


Execution completed in 98.36 seconds

Execution Results for Plans Without HDF Results:


Plan  Success Execution Type
0   01     True  Missing Plans
1   02     True  Missing Plans

## Verification of Results
After executing the plans that were missing HDF results, it's important to verify that the results were properly generated. Let's check if the execution actually created the expected output files.

In [7]:
# Re-initialize the project with the test folder to see updated results
missing_plans_folder = bald_eagle_path.parent / f"{ras.project_name} [MissingPlans]"

if missing_plans_folder.exists():
    # Initialize the project from the test folder
    test_ras = RasPrj()
    init_ras_project(missing_plans_folder, "6.6", ras_object=test_ras)
    
    # Check which plans now have results
    plans_with_results = test_ras.plan_df[test_ras.plan_df['HDF_Results_Path'].notna()]['plan_number'].tolist()
    
    print(f"Plans with results after execution: {', '.join(plans_with_results)}")
    
    # Verify if all previously missing plans now have results
    all_generated = all(plan in plans_with_results for plan in plans_no_results)
    
    if all_generated:
        print("✅ Successfully generated results for all missing plans")
    else:
        print("⚠️ Some plans still don't have results after execution")
        missing_after = [plan for plan in plans_no_results if plan not in plans_with_results]
        print(f"Plans still missing results: {', '.join(missing_after)}")

## Summary of Plan Specification Techniques

In this notebook, we've explored different ways to specify and execute HEC-RAS plans using the RAS Commander library. Here's a summary of the key techniques we've covered:

1. **Basic Plan Specification**
   - Single plan by number: `"01"`
   - List of specific plans: `["01", "03"]`
   - All plans: `ras.plan_df['plan_number'].tolist()`

2. **Advanced Selection**
   - Categorization: Grouping plans by purpose or type
   - Dependencies: Ensuring prerequisite plans are run first
   - Ordered execution: Running plans in a specific sequence

3. **Run Plans with Missing Results (HDF)**
   - Using ras object to determine which plans have results
   - Creating a list of plans with no results
   - Running those plans sequentially

4. NOTE: run_parallel can also run a list of plans, but compute_plan is only made for single plan execution.  


### Best Practices for Plan Specification

1. **Consistent Formatting**: Use two-digit strings for plan numbers ("01" instead of 1)
2. **Descriptive Naming**: Use meaningful short identifiers that describe the plan's purpose
3. **Verify Availability**: Check that specified plans exist before trying to execute them
4. **Document Dependencies**: Keep track of which plans depend on others
5. **Use Appropriate Execution Method**: Choose sequential or parallel based on dependencies and resources
6. **Monitor Performance**: Track execution times to identify optimization opportunities

By applying these techniques, you can create efficient and organized workflows for executing HEC-RAS plans, from simple batch processing to complex dependency-based execution sequences.